# AI Search Connection



In [ ]:
!pip install azure-search-documents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 9.6 MB/s eta 0:00:00


In [ ]:
# AZURE_SEARCH_SERVICE_ENDPOINT=''
# AZURE_SEARCH_INDEX_NAME=''
# AZURE_SEARCH_API_KEY=''

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import os
import uuid

service_endpoint = AZURE_SEARCH_SERVICE_ENDPOINT
index_name = AZURE_SEARCH_INDEX_NAME
key = AZURE_SEARCH_API_KEY

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

In [ ]:
try:
    # Perform a simple search query (empty query lists all documents)
    result = search_client.search(search_text="*", top=1)
    print("Connection successful. Document found:")
    # If result is returned, connection is successful
    for item in result:
        print("Connection successful. Document found:", item["Description"])
        break  # Stop after the first document

except Exception as e:
    print("Connection failed:", e)

Connection successful. Document found:


### UPSERT DATA

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 15.0 MB/s eta 0:00:00


In [ ]:
#Load model
from sentence_transformers import SentenceTransformer

embeddingModel = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Import library
import json
import numpy as np


In [ ]:
# Create document
def createDocument(TableName, Description, DescriptionVector, TableInfo):
  documents = []
  if isinstance(TableInfo, set):
    TableInfo = list(TableInfo)

  for i in range(len(TableName)):
    id = uuid.uuid4()
    docDict = {
        "id": str(id),
        "TableName": TableName[i],
        "Description": Description[i],
        "DescriptionVector": DescriptionVector[i],
        "TableInfo": TableInfo[i]
    }
    documents.append(docDict)

  return documents

# Vectorize

def Vectorize(description):

  descriptionVector = []
  for i in range(len(description)):
    result = embeddingModel.encode(description[i])
    descriptionVector.append(result)

  return descriptionVector



In [ ]:
# Upload document
def uploadDocument(documents):
    for doc in documents:
        for key, value in doc.items():
            if isinstance(value, np.ndarray):
                doc[key] = value.tolist()

    search_client.upload_documents(documents=documents)


# getLastId
def getLastId():
    lastDocument = search_client.search(
        search_text="*",
        order_by=["id desc"],
        top=20,
        include_total_count=True
    )
    for doc in lastDocument:
        if "id" not in doc or doc["id"] is None:
            return 0
        else:
            try:
                return int(doc["id"])
            except ValueError:
                return 0
    return 0
print(getLastId())

0--
0


#### Data

In [ ]:
# Data

TableName = [
    "stations_device",
    "operations_operation_station",
    "operations_operation",
    "stations_station_type",
    "stations_zone",
    "stations_station",
    "stations_parameter",
    "stations_parameter_alarm",
    "maintenances_maintenance_report",
    "maintenances_report_type",
    "maintenances_report_category"
]

Description = [
    "This table stores detailed information about devices installed at each station, including device type, brand, part number, model, and serial number. Each device is linked to a station and can have multiple serial numbers. Devices also hold parameters for triggering alerts, supporting device management and monitoring.",
    "This table logs operational activities at stations, capturing details such as start and end dates, location coordinates, remarks, hydrant number, relocation details, and operation status. Each record is tied to a station and an operation ID, allowing historical and active operations to be tracked effectively.",
    "This table records key details of each operation, including name, status, creation and modification dates, and associated business unit and creator. Each operation is uniquely identified, enabling efficient tracking and management across units.",
    "This table defines the types of stations by storing attributes such as name, category, and color code. It classifies stations into functional groups, supporting organization, analysis, and reporting. It is related to questions about station classification and station types.",
    "This table contains data on geographical zones where stations are located, including zone name and tag. It groups stations based on their geographic location, enabling regional tracking, resource allocation, and performance monitoring across different areas.",
    "This table stores all information for each station, such as station ID, tag, status, location, network status, hydrant number, station type, and commissioning details. It tracks both maintenance and operational statuses, definitely closely related to questions about stations and station relation",
    "This table contains metadata for parameters measured at stations, such as parameter tag, category, unit, data type, and defined limits. Parameters are tied to devices and may have associated alarm settings and chart configurations, supporting performance and alert management.",
    "This table records alarm events triggered by station parameters, logging start and end times, alarm status, description, actual parameter values, and acknowledgment status. It links alarms to specific stations and parameters, helping to manage safety and troubleshooting.",
    "This table stores records of maintenance reports for stations, including timestamps, process status, file paths, report types, and metadata such as grab numbers and failure messages. Each report is linked to a station, providing a detailed history for maintenance-related questions.",
    "This table defines types of maintenance reports by storing the report type name and description. It categorizes maintenance activities, helping to organize and manage different types of reports across stations, and is related to questions about maintenance types.",
    "This table organizes maintenance reports into categories, identified by a unique name. Categories group similar report types, supporting the classification, querying, and analysis of maintenance activities, and are linked to questions about report types and classification."
]

TableInfo = {

    """
CREATE TABLE IF NOT EXISTS public.stations_device
(
    id bigint NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    device_type_name character varying(255),
    device_name character varying(255),
    brand character varying(255),
    part_number character varying(255),
    serial_number character varying(255),
    model character varying(255),
    station_id bigint,
    serial_numbers character varying(255)[],
    CONSTRAINT stations_device_pkey PRIMARY KEY (id),
    CONSTRAINT stations_device_station_id_827ac885_fk_stations_station_id FOREIGN KEY (station_id) REFERENCES public.stations_station (id)
);

INSERT INTO public.stations_device (id, device_type_name, device_name, brand, part_number, serial_number, model, station_id, serial_numbers)
VALUES
(1, 'RTU', 'Tecol', 'iSOLV', 'UFM100', '896503012306004083', 'RTU-800', 165, '{"M1: 896503012306004083"}'),
(2, 'Flow Meter', 'FlowTech', 'iFlow', 'FM300', '896503012306004084', 'FM-500', 165, '{"M2: 896503012306004084"}'),
(3, 'Pressure Sensor', 'PressurePro', 'PTech', 'PS200', '896503012306004085', 'PS-900', 166, '{"M3: 896503012306004085"}'),
    """,

    """

    CREATE TABLE IF NOT EXISTS public.operations_operation_station
(
    id bigint NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    created timestamp with time zone,
    modified timestamp with time zone ,
    location_name character varying(250)  ,
    start_date timestamp with time zone,
    end_date timestamp with time zone,
    remarks character varying(1000),
    high_limit double precision,
    low_limit double precision,
    google_map_link character varying(500) ,
    latitude character varying(250),
    longitude character varying(250),
    operation_id bigint,
    station_id bigint,
    hydrant_number double precision,
    relocate_by_id bigint,
    relocate_date timestamp with time zone,
    relocate_end_date timestamp with time zone,
    operation_station_code character varying(50),
    is_cancelled boolean ,
    CONSTRAINT operations_operation_station_pkey PRIMARY KEY (id),
    CONSTRAINT operations_operation_operation_id_b36b8226_fk_operation FOREIGN KEY (operation_id) REFERENCES public.operations_operation (id),
    CONSTRAINT operations_operation_station_id_fb1c684b_fk_stations_ FOREIGN KEY (station_id) REFERENCES public.stations_station (id)
)

INSERT INTO public.operations_operation_station ( id, created, modified,, start_date, end_date, remarks, high_limit, low_limit, google_map_link, latitude, longitude, operation_id, station_id, hydrant_number, relocate_by_id, relocate_date, relocate_end_date, operation_station_code, is_cancelled )
VALUES
(1, '2024-10-11 08:00:00+00', '2024-10-11 09:00:00+00', ‘Bedeck’, '2024-10-11 08:30:00+00', '2024-10-12 08:30:00+00', 'Initial setup for operation', 75.5, 30.2, 'https://maps.google.com/?q=10.762622,106.660172', 'nyp0/Nf97jwoEbud3MFvAQ==', 'nzR03+rB6TsvFrya28ZoBg==', 1, 101, 5.0, 6, NULL, NULL, 'MOBM-MF-24-SEP-0043', FALSE),

(2, '2024-10-10 10:00:00+00', '2024-10-10 12:00:00+00', ‘Tampines’, '2024-10-10 11:00:00+00', '2024-10-11 11:00:00+00', 'Routine maintenance', 80.0, 25.0, 'https://maps.google.com/?q=10.823099,106.629664', 'nyp0/Nf97jwoEbud3MFvAQ==', 'nzR03+rB6TsvFrya28ZoBg==', 2, 102, 6.0, 2001, '2024-10-15 08:00:00+00', '2024-10-15 10:00:00+00', 'MOBM-MF-24-AUG-0023', TRUE),

(3, '2024-09-30 14:00:00+00', '2024-09-30 15:00:00+00', ‘fungal’, '2024-09-30 14:30:00+00', '2024-10-01 14:30:00+00', 'Emergency repair', 85.5, 28.7, 'https://maps.google.com/?q=10.841865,106.732784', 'nyp0/Nf97jwoEbud3MFvAQ==', 'nzR03+rB6TsvFrya28ZoBg==', 3, 103, 7.5, 98,  '2024-10-15 08:00:00+00', '2024-10-15 10:00:00+00', 'MOBM-MWQPA-24-AUG-0022', FALSE),
    """,

    """
    CREATE TABLE IF NOT EXISTS public.operations_operation
(
    id bigint NOT NULL GENERATED BY DEFAULT AS IDENTITY ,
    created timestamp with time zone,
    modified timestamp with time zone,
    name character varying(255),
    operation_status character varying(50) ,
    created_by_id bigint,
    business_unit_id bigint,
    CONSTRAINT operations_operation_pkey PRIMARY KEY (id),
    CONSTRAINT operations_operation_business_unit_id_55f929c1_fk_business_ FOREIGN KEY (business_unit_id) REFERENCES public.business_units_business_unit (id)
    CONSTRAINT operations_operation_created_by_id_ce1ff916_fk_users_user_id FOREIGN KEY (created_by_id) REFERENCES public.users_user (id)
)

class OperationStatus(models.TextChoices):
    TO_DO = "to_do",
    IN_PROGRESS = "in_progress",
    DONE = "done",
    CANCEL = "cancel"

INSERT INTO public.operations_operation (created, modified, name, operation_status, created_by_id, business_unit_id)
VALUES
(2024-06-07 14:11:16.392636, 2024-06-11 11:50:56.458504, 'Operation015', 'in_progress', 1, 1),
(2024-06-07 14:11:16.392636, 2024-06-11 11:50:56.458504, 'Operation016', 'to_do', 2, 2),
(2024-06-07 14:11:16.392636, 2024-06-11 11:50:56.458504, 'Operation017', 'done', 3, 3),
(2024-06-07 14:11:16.392636, 2024-06-11 11:50:56.458504, 'Operation018', 'cancel', 4, 4),

    """,


    """
CREATE TABLE IF NOT EXISTS public.stations_station_type
(
    id bigint NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    name character varying(255),
    station_category character varying(20),
    color character varying(50) ,
    CONSTRAINT stations_station_type_pkey PRIMARY KEY (id)
);

INSERT INTO public.stations_station_type (id, name, station_category, color)
VALUES
(1, 'WQPA', 'fixed', '#FE3401'),
(2, 'FPA', 'fixed', '#81C9FF'),
(3, 'FF', 'fixed', '#51D8BA'),

class StationCategory(models.TextChoices):
    FIXED = "fixed",
    MOBILE = "mobile",

    """,
    """
CREATE TABLE IF NOT EXISTS public.stations_zone
(
    id bigint NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    name character varying(255),
    zone_tag character varying(255),
    CONSTRAINT stations_zone_pkey PRIMARY KEY (id)
);

INSERT INTO public.stations_zone (id, name, zone_tag)
VALUES
(1, 'BPSR', 'BPSR'),
(2, 'NYSR', 'NYSR'),
(3, 'EASTERN', 'EASTERN'),

    """,
    """
CREATE TABLE IF NOT EXISTS public.stations_station
(
    id bigint NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    created timestamp with time zone,
    modified timestamp with time zone,
    station_id character varying(150),
    station_tag character varying(255) ,
    station_status character varying(50),
    network_status character varying(50),
    latitude character varying(250),
    longitude character varying(250),
    location character varying(250),
    hydrant_number character varying(250),
    created_by_id bigint,
    station_type_id bigint,
    zone_id bigint,
    power_supply_type character varying(50),
    commissioned boolean NOT NULL,
    code_number character varying(50),
    existing_id character varying(250),
    lamp_pole_number character varying(250),
    area character varying(250),
    pipeline_size character varying(50),
    cm_status boolean,
    cm_last_record_time timestamp with time zone ,
    ready_for_booking boolean,
    CONSTRAINT stations_station_pkey PRIMARY KEY (id),
    CONSTRAINT stations_station_station_id_3c342313_uniq UNIQUE (station_id),
    CONSTRAINT stations_station_station_type_id_9bdc36cd_fk_stations_ FOREIGN KEY (station_type_id) REFERENCES public.stations_station_type (id)
    CONSTRAINT stations_station_zone_id_9d879fcf_fk_stations_zone_id FOREIGN KEY (zone_id) REFERENCES public.stations_zone (id)
);

INSERT INTO public.stations_station ( id, created, modified, station_id, station_tag, station_status, network_status, latitude, longitude, location, hydrant_number, created_by_id, station_type_id, zone_id, power_supply_type, commissioned, code_number, existing_id, lamp_pole_number, area, pipeline_size, cm_status, cm_last_record_time, ready_for_booking )
VALUES
(1, '2024-10-11 08:00:00+00', '2024-10-11 09:00:00+00', 'WQPA-700-06', 'Station A', 'fully_operational', 'online', 'nyp0/Nf97jwoEbud3MFvAQ==', 'nzR03+rB6TsvFrya28ZoBg==', 'District 1, Ho Chi Minh City', 'HYD001', 1, 1, 1, 'solar', TRUE, '700-06', 'BKSR_028', 'LP001', 'west’, '200’, TRUE, '2024-10-10 08:00:00+00', TRUE),

(2, '2024-09-11 08:00:00+00', '2024-09-11 09:00:00+00', 'WQPA-BKSR-42', 'Station B', 'under_maintenance', 'offline', 'nyp0/Nf97jwoEbud3MFvAQ==', 'nzR03+rB6TsvFrya28ZoBg==', 'District 7, Ho Chi Minh City', 'HYD002', 2, 2, 2, 'lpt', FALSE, '700-06', 'BKSR_028', 'LP002', ‘east’, '157, FALSE, '2024-09-10 08:00:00+00', FALSE),

(3, '2024-08-11 08:00:00+00', '2024-08-11 09:00:00+00', 'PA-ISR-01', 'Station C', 'under_maintenance', 'maintenance', 'nyp0/Nf97jwoEbud3MFvAQ==', 'nzR03+rB6TsvFrya28ZoBg==', 'Thu Duc, Ho Chi Minh City', 'HYD003', 3, 3, 3, 'lpt', TRUE, '700-06', 'BKSR_028', 'LP003', ’sounth, '300’, TRUE, '2024-08-10 08:00:00+00', TRUE),

class StationStatus(models.TextChoices):
    FULLY_OPERATIONAL = "fully_operational", _("Fully_operational")
    UNDER_MAINTENANCE = "under_maintenance", _("Under_maintenance")
    REQUIRE_ATTENTION = "under_maintenance", _("Require_attention")
    FAILED = "failed", _("Failed")

class NetworkStatus(models.TextChoices):
    ONLINE = "online", _("Online")
    OFFLINE = "offline", _("Offline")
    MAINTENANCE = "maintenance", _("Maintenance")
    INTRUDER = "intruder", _("Intruder")

class PowerSupplyType(models.TextChoices):
    SOLAR = "solar", _("Solar")
    LPT = "lpt", _("Lpt")

    """,
    """
CREATE TABLE IF NOT EXISTS public.stations_parameter
(
    id bigint NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    parameter_tag character varying(255) ,
    category character varying(255),
    unit character varying(255),
    data_type character varying(255),
    max_value double precision,
    min_value double precision,
    high_limit double precision,
    low_limit double precision,
    tag_type character varying(20),
    have_chart boolean,
    alarm_enabled boolean,
    device_id bigint,
    "offset" double precision,
    parameter_name character varying(255),
    roc_period double precision,
    roc_span character varying(255),
    CONSTRAINT stations_parameter_pkey PRIMARY KEY (id),
    CONSTRAINT stations_parameter_device_id_89cd140c_fk_stations_device_id FOREIGN KEY (device_id) REFERENCES public.stations_device (id)
);

INSERT INTO public.stations_parameter ( id, parameter_tag, category, unit, data_type, max_value, min_value, high_limit, low_limit, tag_type, have_chart, alarm_enabled, device_id, "offset", parameter_name, roc_period, roc_span )
VALUES
(1, 'Temp001', 'meas', 'Celsius', 'float', 100.0, -20.0, 90.0, 0.0, 'alarm', TRUE, TRUE, 132, 0.0, 'pH', 10.0, 'day'),
(2, 'Pressure001', 'meas', 'Pascal', 'float', 5000.0, 0.0, 4500.0, 500.0, 'alarm', TRUE, TRUE, 12, 0.0, 'Maintenance', 5.0, 'hour'),
(3, 'Flow001', ‘meas’, 'L/min', 'float', 200.0, 10.0, 180.0, 20.0, 'physical', TRUE, FALSE, 333, 0.5, 'Conductivity', 15.0, 'minute'),

class DataType(models.TextChoices):
    STRING = "string", _("String")
    BOOLEAN = "boolean", _("Boolean")
    FLOAT = "float", _("Float")

class TagType(models.TextChoices):
    PHYSICAL = "physical", _("Physical")
    VIRTUAL = "virtual", _("Virtual")
    LOGICAL = "logical", _("Logical")
    ALARM = "alarm", _("Alarm")

    """,
    """
CREATE TABLE IF NOT EXISTS public.stations_parameter_alarm
(
    id bigint NOT NULL GENERATED BY DEFAULT AS IDENTITY ,
    created timestamp with time zone,
    modified timestamp with time zone,
    deleted timestamp with time zone,
    deleted_by_cascade boolean,
    parameter_alarm_status character varying(20),
    description character varying(500),
    start_time timestamp with time zone,
    end_time timestamp with time zone,
    created_by_name character varying(100),
    alarm_type character varying(20),
    created_by_id bigint,
    modified_by_id bigint,
    parameter_id bigint,
    reported_date timestamp with time zone,
    parameter_alarm_type character varying(20),
    actual_value double precision,
    station_group_id bigint,
    is_active boolean NOT NULL,
    parameter_tag character varying(255),
    station_id character varying(150),
    is_acknowledged boolean,
    acknowledged_reason text,
    CONSTRAINT stations_parameter_alarm_pkey PRIMARY KEY (id),
    CONSTRAINT stations_parameter_a_parameter_id_e81ffea8_fk_stations_ FOREIGN KEY (parameter_id) REFERENCES public.stations_parameter (id) M
)

INSERT INTO public.stations_parameter_alarm ( id, created, modified, deleted, deleted_by_cascade, parameter_alarm_status, description, start_time, end_time, created_by_name, alarm_type, created_by_id, modified_by_id, parameter_id, reported_date, parameter_alarm_type, actual_value, station_group_id, is_active, parameter_tag, station_id, is_acknowledged, acknowledged_reason ) VALUES
(1, '2024-10-11 08:00:00+00', '2024-10-11 09:00:00+00', NULL, FALSE, 'pending', 'High temperature detected', '2024-10-11 08:15:00+00', NULL, 'John Doe', 'threshold', 1, 2, 1, '2024-10-11 08:00:00+00', 'user_alarm', 95.5, 1, TRUE, 'pH', 'WQPA-BKSR-45', FALSE, NULL),

(2, '2024-09-11 08:00:00+00', '2024-09-11 09:00:00+00', 2024-09-11 09:00:00+00, FALSE, 'complete', 'Low pressure detected', '2024-09-11 08:15:00+00', '2024-09-11 09:00:00+00', 'Jane Smith', 'threshold', 2, 3, 2, '2024-09-11 08:00:00+00', 'system_alarm', 5.0, 2, FALSE, 'battery', 'WQPA-BKSR-45', TRUE, 'Issue resolved by resetting the system'),

(3, '2024-08-11 08:00:00+00', '2024-08-11 09:00:00+00', NULL, FALSE, 'cancel', 'High flow rate detected', '2024-08-11 08:30:00+00', NULL, 'Emily Johnson', 'anomaly', 3, 4, 3, '2024-08-11 08:00:00+00', 'station_alarm', 210.0, 3, TRUE, 'acoustic', 'WQPA-BKSR-45', FALSE, NULL);

class ParameterAlarmStatus(models.TextChoices):
    PENDING = "pending", _("Pending")
    COMPLETE = "complete", _("Complete")
    CANCEL = "cancel", _("Cancel")

class ParameterAlarmType(models.TextChoices):
    USER_ALARM = "user_alarm", _("User_alarm")
    SYSTEM_ALARM = "system_alarm", _("System_alarm")
    STATION_ALARM = "station_alarm", _("Station_alarm")

    """,
    """
CREATE TABLE IF NOT EXISTS public.maintenances_maintenance_report (
    id BIGINT NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    deleted BOOLEAN DEFAULT FALSE,
    deleted_by_cascade BOOLEAN DEFAULT FALSE,
    created TIMESTAMPTZ NOT NULL,
    modified TIMESTAMPTZ NOT NULL,
    reported_date TIMESTAMPTZ,
    file_path VARCHAR(255),
    report_file VARCHAR(255),
    data_path VARCHAR(255),
    process_status VARCHAR(50),
    created_by_id BIGINT,
    report_type_id BIGINT,
    station_id BIGINT,
    generate_fail_message TEXT,
    grab_number VARCHAR(50),
    next_maintenance_date TIMESTAMPTZ,
    end_datetime TIMESTAMPTZ,
    last_maintenance_date TIMESTAMPTZ,
    start_datetime TIMESTAMPTZ,
    CONSTRAINT maintenances_maintenance_report_pkey PRIMARY KEY (id),
    CONSTRAINT maintenances_maintenance_report_report_type_id_fkey FOREIGN KEY (report_type_id) REFERENCES public.maintenances_report_type (id)
);


INSERT INTO public.maintenances_maintenance_report ( id, deleted, deleted_by_cascade, created, modified, reported_date, file_path, report_file, data_path, process_status, created_by_id, report_type_id, station_id, generate_fail_message, grab_number, next_maintenance_date, end_datetime, last_maintenance_date, start_datetime )

VALUES

(1, 024-09-30 15:00:00+00, FALSE, '2024-10-11 08:00:00+00', '2024-10-11 09:00:00+00', '2024-10-11 07:00:00+00', '/files/reports/report1.pdf', 'report1.pdf', '/data/reports/data1', ‘done’, 1, 1, 101, NULL, 'GRAB-001', '2025-10-11 08:00:00+00', '2024-10-11 12:00:00+00', '2023-10-11 08:00:00+00', '2024-10-11 08:00:00+00'),

(2, 024-09-30 15:00:00+00, FALSE, '2024-09-11 08:00:00+00', '2024-09-11 09:00:00+00', '2024-09-11 07:00:00+00', '/files/reports/report2.pdf', 'report2.pdf', '/data/reports/data2', 'pending', 2, 2, 102, 'Error generating report', 'GRAB-002', '2025-09-11 08:00:00+00', NULL, '2023-09-11 08:00:00+00', '2024-09-11 08:00:00+00'),

(3, 024-09-30 15:00:00+00, FALSE, '2024-08-11 08:00:00+00', '2024-08-11 09:00:00+00', '2024-08-11 07:00:00+00', '/files/reports/report3.pdf', 'report3.pdf', '/data/reports/data3', 'fail', 3, 3, 103, 'Network failure during report generation', 'GRAB-003', '2025-08-11 08:00:00+00', '2024-08-11 12:00:00+00', '2023-08-11 08:00:00+00', '2024-08-11 08:00:00+00'),

class MaintenanceReportProcessStatus(models.TextChoices):
    PENDING = "pending",
    DONE = "done",
    FAIL = "fail",

    """,
    """
CREATE TABLE IF NOT EXISTS public.maintenances_report_type (
    id BIGINT NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    name VARCHAR(255) NOT NULL,
    description TEXT,
    report_category_id BIGINT,
    CONSTRAINT maintenances_report_type_pkey PRIMARY KEY (id),
    CONSTRAINT maintenances_report_type_report_category_id_fkey FOREIGN KEY (report_category_id) REFERENCES public.maintenances_report_category (id)
);

INSERT INTO public.maintenances_report_type (id, name, description, report_category_id)
VALUES
(1, ‘’CM, 'Corrective Maintenance (Grab Sample)', 1),
(2, ‘PM’, 'Preventive Maintenance (Grab Sample Monthly)', 2),
(3, 'Calibration', 'Preventive Maintenance (Calibration 6th Monthly)', 3),

    """,
    """
CREATE TABLE IF NOT EXISTS public.maintenances_report_category (
    id BIGINT NOT NULL GENERATED BY DEFAULT AS IDENTITY,
    name VARCHAR(255) NOT NULL,
    CONSTRAINT maintenances_report_category_pkey PRIMARY KEY (id)
);

INSERT INTO public.maintenances_report_category (id, name)
VALUES
(1, 'Safety'),
(2, 'Performance'),
(3, 'Inspection'),
(4, 'Incident'),
(5, 'Maintenance');

    """,
}


### Upload

In [ ]:
# Upload data

DescriptionVector = Vectorize(Description)

# LastId = getLastId()

document = createDocument(TableName, Description, DescriptionVector, TableInfo)

uploadDocument(document)

### TEST

In [ ]:
def createSearchRequest(search_text, vector, top_k=7, fields="DescriptionVector", select_fields="HotelName, Description", count=True):
    search_request = {
        "count": count,
        "search": search_text,
        "select": "TableName",
        "top": top_k,
        "vectorQueries": [
            {
                "vector": vector,
                "k": top_k,
                "fields": fields,
                "kind": "vector",
                "exhaustive": True
            }
        ]
    }
    return search_request

# Example usage:
search_text = "show me the most recent maintenance report for station BBSR_12."
vector = embeddingModel.encode(search_text)  # Example vector (replace with your actual vector)
top_k = 7

search_query = createSearchRequest(search_text, vector, top_k)



In [ ]:
search_results = search_client.search(
    search_text=search_text,
    vector={
        "value": vector,
        "fields": "DescriptionVector",
        "k": 5
    },
    select="TableName",
    top=5
)

In [ ]:
import time
from azure.search.documents.models import VectorizedQuery

# Start the timer
start_time = time.time()

# Your query logic
search_text = "which are the stations offline in zone ABCD for the last 24 hours?"
vector = embeddingModel.encode(search_text)

r = search_client.search(None, top=10, vector_queries=[VectorizedQuery(vector=vector,  fields="DescriptionVector")])

# Print the results
for doc in r:
    content = doc["TableName"].replace("\n", " ")[:150]
    tableInfo = doc["TableInfo"].replace("\n", " ")
    id = doc["id"]
    print(f"score: {id}, {doc['@search.score']}. {tableInfo}")

# End the timer
end_time = time.time()

# Calculate and print the running time
running_time = end_time - start_time
print(f"Running time: {running_time} seconds")


In [ ]:
def cosine_similarity(vec1, vec2):
    # Convert input vectors to NumPy arrays
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Compute the dot product and magnitudes of the vectors
    dot_product = np.dot(vec1, vec2)
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)

    # Compute the cosine similarity
    if magnitude_vec1 == 0 or magnitude_vec2 == 0:
        return 0.0
    cosine_sim = dot_product / (magnitude_vec1 * magnitude_vec2)

    return cosine_sim

from IPython.display import display, HTML
results = []

for i in range(len(DescriptionVector)):
    percent = cosine_similarity(DescriptionVector[i], vector)
    results.append((TableName[i], percent))

sorted_results = sorted(results, key=lambda x: x[1], reverse=True)

for item in sorted_results:
    display(HTML(f"<b>{item[0]}</b>: {item[1]}"))

### test line break

In [ ]:
# AZURE_SEARCH_SERVICE_ENDPOINT=''
# AZURE_SEARCH_INDEX_NAME=''
# AZURE_SEARCH_API_KEY=''

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import os

service_endpoint = AZURE_SEARCH_SERVICE_ENDPOINT
index_name = AZURE_SEARCH_INDEX_NAME
key = AZURE_SEARCH_API_KEY

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_list = vector.tolist()
docDict = {
        "id": "11",
        "TableName": "Hello",
        "Description": "M1: 896503012306004083\nAnotherLine",
        "DescriptionVector": vector_list,
        "TableInfo": "M1: 896503012306004083\nAnotherLine"
    }

# Upsert document
result = search_client.upload_documents(documents=[docDict])

# Check the result
if result[0].succeeded:
    print(f"Document with ID {docDict['id']} was upserted successfully.")
else:
    print(f"Failed to upsert document with ID {docDict['id']}.")


In [ ]:
import time
from azure.search.documents.models import VectorizedQuery

# Start the timer
start_time = time.time()

# Your query logic
search_text = "which are the stations offline in zone ABCD for the last 24 hours?"
vector = embeddingModel.encode(search_text)

r = search_client.search(None, top=11, vector_queries=[VectorizedQuery(vector=vector,  fields="DescriptionVector")])

# Print the results
for doc in r:
    content = doc["TableName"].replace("\n", " ")[:150]
    tableInfo = doc["TableInfo"].replace("\\n", "\n")
    print(f"score: {doc['@search.score']}. {tableInfo}")

# End the timer
end_time = time.time()

# Calculate and print the running time
running_time = end_time - start_time
print(f"Running time: {running_time} seconds")


# AI Search Related question

In [ ]:
!pip install azure-search-documents

In [ ]:
AZURE_SEARCH_SERVICE_ENDPOINT=''
AZURE_SEARCH_INDEX_NAME=''
AZURE_SEARCH_API_KEY=''

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
import os
import uuid
import numpy as np

service_endpoint = AZURE_SEARCH_SERVICE_ENDPOINT
index_name = AZURE_SEARCH_INDEX_NAME
key = AZURE_SEARCH_API_KEY

search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))

In [ ]:
try:
    # Perform a simple search query (empty query lists all documents)
    result = search_client.search(search_text="*", top=1)
    print("Connection successful. Document found:")
    # If result is returned, connection is successful
    for item in result:
        print("Connection successful. Document found:", item["Question"])
        break  # Stop after the first document

except Exception as e:
    print("Connection failed:", e)

Connection successful. Document found:


## UPSERT DATA

In [ ]:
!pip install -U sentence-transformers

In [ ]:
#Load model
from sentence_transformers import SentenceTransformer

embeddingModel = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Data


In [ ]:
Question = [
    "What are the stations with pressure alarms in zone WQPA-MNSR-21 for the past year?",
    "How many WQPA stations are there in zone KRSR?",
    "what is the commissioned date for station MWQPA-600-17?",
    "when was WQPA-BKSR-51 station oldest maintained? [last maintenance date]",
    "How many alarms were there for MWQPA-600-100-OTA for the last 2 years?",
    "show me the most recent maintenance report for station MWQPA-600-100-OTA."
]

SQLquery = [
    """
select pa.station_id, pa.end_time
from stations_parameter_alarm pa
where pa.station_id = 'WQPA-MNSR-21'
and pa.parameter_tag = 'pressure'
and pa.end_time >= (NOW() AT TIME ZONE 'UTC' + INTERVAL '8 hours') - INTERVAL '1 years';

""",
    """
select count(ss.id)
from stations_station ss
join stations_zone sz on sz.id = ss.zone_id
join stations_station_type st on st.id = ss.station_type_id
where sz.name = 'KRSR'
and st.name ='WQPA';

    """
    ,
    """
SELECT mr.reported_date
FROM stations_station ss
JOIN maintenances_maintenance_report mr ON mr.station_id = ss.station_id
WHERE ss.station_id = 'MWQPA-600-17'
ORDER BY mr.reported_date DESC
LIMIT 1;

    """,
    """
SELECT mr.reported_date
FROM stations_station ss
JOIN maintenances_maintenance_report mr ON mr.station_id = ss.station_id
WHERE ss.station_id = 'MWQPA-600-17'
ORDER BY mr.reported_date DESC
LIMIT 1;
    """
    ,
    """
select count(mr.id)
from stations_station ss
join maintenances_maintenance_report mr on mr.station_id = ss.id
where ss.station_id = 'MWQPA-600-100-OTA'
AND mr.end_datetime >= (NOW() AT TIME ZONE 'UTC' + INTERVAL '8 hours') - INTERVAL '2 years';
    """
    ,
    """
select mr.file_path
from stations_station ss
join maintenances_maintenance_report mr on mr.station_id = ss.id
where ss.station_id = 'MWQPA-600-100-OTA'
order by mr.reported_date asc
limit 1;

    """
]

In [ ]:
# Create documents
def createQuestionDocument(question, sqlquery):
  documents = []
  for i in range(len(question)):
    id = uuid.uuid4()
    vectorQuestion = embeddingModel.encode(question[i])
    docDict = {
        "id": str(id),
        "Question": question[i],
        "QuestionVector": vectorQuestion.tolist(),
        "Query": sqlquery[i]
    }
    documents.append(docDict)

  return documents

# Upload document
def uploadDocument(documents):
    for doc in documents:
        for key, value in doc.items():
            if isinstance(value, np.ndarray):
                doc[key] = value.tolist()  # Ensure all numpy arrays are lists

    try:
        response = search_client.upload_documents(documents=documents)
        print("Upload successful:", response)
    except Exception as e:
        print("HttpResponseError:", e)
        print("Code:", e.status_code)
        print("Message:", e.message)
        print("Response:", e.response)


In [ ]:
documents = createQuestionDocument(Question, SQLquery)

In [ ]:
# Upload document

uploadDocument(documents)

Upload successful: [<azure.search.documents._generated.models._models_py3.IndexingResult object at 0x79c5df116b30>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x79c5df116e60>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x79c5df1166e0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x79c5df117100>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x79c5df117ac0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x79c5df1179d0>]


## Connect to Postgres

In [ ]:
import psycopg2
query = """
select count(ss.id) from stations_station ss join stations_zone sz on sz.id = ss.zone_id where sz.name = 'KRSR';
"""
# Database connection details
DB_HOST = "c-uatenvdb.nhzdsmvyl3u7ws.postgres.cosmos.azure.com"
DB_NAME = "citus"
DB_PORT = "5432"
POSTGRES_USER = "citus"
POSTGRES_PASSWORD = "uatDB123478@!"
SSLMODE = "require"

# Connect to the PostgreSQL database
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=POSTGRES_USER,
        password=POSTGRES_PASSWORD,
        port=DB_PORT,
        sslmode=SSLMODE
    )
    cursor = conn.cursor()

    # Sample query
    query = query
    cursor.execute(query)

    # Fetch and print the results
    rows = cursor.fetchall()
    print(rows)

    # Close the cursor and connection
    cursor.close()
    conn.close()

except Exception as e:
    print("An error occurred:", e)

[(25,)]
